In [ ]:
pip install ipywidgets


  Using cached jedi-0.19.1-py2.py3-none-any.whl.metadata (22 kB)
Using cached jedi-0.19.1-py2.py3-none-any.whl (1.6 MB)


In [5]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, IntSlider

# Function to model Susceptible, Infected, and Recovered populations using a probabilistic infection model
def simulate_infection_graph(population_size, initial_infected, R0, recovery_rate, contagious_factor_for_iso, contagious_factor_for_uniso, isolation_rate, days, vaccination_rate, vaccine_efficacy):

    # Vaccinated and unvaccinated population split
    vaccinated_population = population_size * vaccination_rate
    unvaccinated_population = population_size * (1 - vaccination_rate)

    # Initialize arrays to store the number of infected, recovered, and susceptible individuals for each day
    susceptible_vaccinated = [vaccinated_population - initial_infected * vaccination_rate]
    susceptible_unvaccinated = [unvaccinated_population - initial_infected * (1 - vaccination_rate)]

    infected_vaccinated = [initial_infected * vaccination_rate]
    infected_unvaccinated = [initial_infected * (1 - vaccination_rate)]

    recovered_vaccinated = [0]
    recovered_unvaccinated = [0]

    for day in range(1, days + 1):
        # Calculate new infections for vaccinated and unvaccinated populations
        new_infected_vaccinated = (infected_vaccinated[-1] * R0 * vaccine_efficacy) * (susceptible_vaccinated[-1] / population_size) * (1 - isolation_rate * contagious_factor_for_iso)
        new_infected_unvaccinated = (infected_unvaccinated[-1] * R0) * (susceptible_unvaccinated[-1] / population_size) * (1 - isolation_rate * contagious_factor_for_uniso)

        # Calculate new recoveries for both populations
        new_recovered_vaccinated = infected_vaccinated[-1] * recovery_rate
        new_recovered_unvaccinated = infected_unvaccinated[-1] * recovery_rate

        # Update susceptible, infected, and recovered populations for both groups
        susceptible_vaccinated.append(susceptible_vaccinated[-1] - new_infected_vaccinated)
        susceptible_unvaccinated.append(susceptible_unvaccinated[-1] - new_infected_unvaccinated)

        infected_vaccinated.append(infected_vaccinated[-1] + new_infected_vaccinated - new_recovered_vaccinated)
        infected_unvaccinated.append(infected_unvaccinated[-1] + new_infected_unvaccinated - new_recovered_unvaccinated)

        recovered_vaccinated.append(recovered_vaccinated[-1] + new_recovered_vaccinated)
        recovered_unvaccinated.append(recovered_unvaccinated[-1] + new_recovered_unvaccinated)

    # Combine vaccinated and unvaccinated populations for total
    total_susceptible = np.array(susceptible_vaccinated) + np.array(susceptible_unvaccinated)
    total_infected = np.array(infected_vaccinated) + np.array(infected_unvaccinated)
    total_recovered = np.array(recovered_vaccinated) + np.array(recovered_unvaccinated)

    # Plotting the results
    plt.figure(figsize=(10, 6))

    # Plot susceptible population
    plt.plot(range(days + 1), total_susceptible, label='Susceptible Population', color='orange')

    # Plot infected population
    plt.plot(range(days + 1), total_infected, label='Infected Population', color='red')

    # Plot recovered population
    plt.plot(range(days + 1), total_recovered, label='Recovered Population', color='green')

    # Adding labels, title, and legend for better understanding
    plt.title(f'Infection Simulation: Susceptible, Infected, and Recovered Populations')
    plt.xlabel('Days')
    plt.ylabel('Population')
    plt.legend()
    plt.grid(True)

    # Show the plot
    plt.tight_layout()
    plt.show()

# Interactive sliders for the simulation parameters
interact(simulate_infection_graph,
         population_size=IntSlider(min=50, max=500, step=10, value=100, description='Population Size'),
         initial_infected=IntSlider(min=1, max=50, step=1, value=5, description='Initial Infected'),
         R0=FloatSlider(min=0.5, max=5.0, step=0.1, value=2.0, description='R0 (Infection Rate)'),
         recovery_rate=FloatSlider(min=0.01, max=1.0, step=0.01, value=0.1, description='Recovery Rate'),
         contagious_factor_for_iso=FloatSlider(min=0.01, max=1.0, step=0.01, value=0.1, description='Contagious Factor (Isolated)'),
         contagious_factor_for_uniso=FloatSlider(min=0.01, max=1.0, step=0.01, value=0.3, description='Contagious Factor (Unisolated)'),
         isolation_rate=FloatSlider(min=0.0, max=1.0, step=0.01, value=0.5, description='Isolation Rate'),
         days=IntSlider(min=1, max=20, step=1, value=10, description='Days'),
         vaccination_rate=FloatSlider(min=0.0, max=1.0, step=0.01, value=0.7, description='Vaccination Rate'),
         vaccine_efficacy=FloatSlider(min=0.5, max=1.0, step=0.01, value=0.9, description='Vaccine Efficacy'));


interactive(children=(IntSlider(value=100, description='Population Size', max=500, min=50, step=10), IntSlider…